In [1]:
import tensorflow

In [2]:
tensorflow.__version__

'2.3.0'

In [3]:
sample_text_1="bitty bought a bit of butter"
sample_text_2="but the bit of butter was a bit bitter"
sample_text_3="so she bought some better butter to make the bitter butter better"

In [22]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
from nltk import word_tokenize

In [4]:
corpus = [sample_text_1, sample_text_2, sample_text_3]

In [13]:
corpus

['bitty bought a bit of butter',
 'but the bit of butter was a bit bitter',
 'so she bought some better butter to make the bitter butter better']

In [17]:
corpus = [word_tokenize(x) for x in corpus]

In [18]:
corpus

[['bitty', 'bought', 'a', 'bit', 'of', 'butter'],
 ['but', 'the', 'bit', 'of', 'butter', 'was', 'a', 'bit', 'bitter'],
 ['so',
  'she',
  'bought',
  'some',
  'better',
  'butter',
  'to',
  'make',
  'the',
  'bitter',
  'butter',
  'better']]

In [20]:
from nltk.corpus import  stopwords

In [23]:
stop_words = set(stopwords.words('english'))

In [24]:
print(stop_words)

{'which', 'above', 'we', "she's", 'who', 'into', 'further', 'hadn', 'now', 'needn', 'doesn', 'mightn', 'if', 'against', 'has', 'or', "mightn't", 'about', 'most', 'ourselves', 'all', 'y', 'too', 'own', 'with', 'there', "shouldn't", 'out', "won't", 'ain', "you've", 'had', 'don', 've', 'do', 'being', 'didn', 'ma', 'whom', 'have', 'the', "you're", 'his', "didn't", 'its', 'such', 'o', 'what', 'is', 'won', "don't", 'nor', 'once', 'more', 'between', 'was', 'until', 'myself', 'wasn', "should've", 'then', "haven't", 'her', 'ours', 'up', 'your', 'our', 'why', 'itself', 'yourself', 'it', 'those', 'should', 'of', 't', 'off', 'aren', 'to', 'and', 'as', 'themselves', "you'll", 'does', 'him', 'on', 'my', 'can', 'just', 'very', 'theirs', "that'll", 'yourselves', 'during', 'how', 'i', 'am', 'some', 'than', "hadn't", 'down', 're', "it's", 'other', 'at', 'these', 'while', 'this', 'having', 'they', 'no', 'himself', 'both', "mustn't", 'here', 'where', 'so', 'couldn', 'been', 'an', 'be', 'did', 'were', 'any

In [42]:
corpusClean = []
for doc in corpus:
  arr = []
  for word in doc:
    if word not in stop_words:
      arr.append(word)
  corpusClean.append(arr)
print(corpusClean)

[['bitty', 'bought', 'bit', 'butter'], ['bit', 'butter', 'bit', 'bitter'], ['bought', 'better', 'butter', 'make', 'bitter', 'butter', 'better']]


In [27]:
maxLen = -1
for doc in corpusClean:
  val = len(doc)
  if maxLen < val:
    maxLen = val  

In [28]:
print(maxLen)

7


In [43]:
corpusClean

[['bitty', 'bought', 'bit', 'butter'],
 ['bit', 'butter', 'bit', 'bitter'],
 ['bought', 'better', 'butter', 'make', 'bitter', 'butter', 'better']]

In [5]:
from tensorflow.keras import layers

In [7]:
from tensorflow.keras.preprocessing.text import one_hot 

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [30]:
corpusOneHot

[[3, 42, 21, 24, 17, 42],
 [38, 41, 24, 17, 42, 26, 21, 24, 40],
 [32, 23, 42, 47, 16, 42, 5, 42, 41, 40, 42, 16]]

In [49]:
maxVocab = 50
corpusClean = [' '.join(x) for x in corpusClean] 
corpusOneHot = [one_hot(x,maxVocab) for x in corpusClean]

In [50]:
corpusClean

['bitty bought bit butter',
 'bit butter bit bitter',
 'bought better butter make bitter butter better']

In [51]:
corpusOneHot

[[3, 42, 24, 42], [24, 42, 24, 40], [42, 16, 42, 42, 40, 42, 16]]

In [53]:
corpusSequence = pad_sequences(corpusOneHot,maxlen=7,padding='post')

In [54]:
corpusSequence

array([[ 3, 42, 24, 42,  0,  0,  0],
       [24, 42, 24, 40,  0,  0,  0],
       [42, 16, 42, 42, 40, 42, 16]], dtype=int32)

In [56]:
word_input = layers.Input(shape=(7,),dtype='float64')

In [57]:
word_input

<tf.Tensor 'input_2:0' shape=(None, 7) dtype=float64>

In [60]:
x = layers.Embedding(50,5)(word_input)

In [61]:
x

<tf.Tensor 'embedding_1/embedding_lookup/Identity_1:0' shape=(None, 7, 5) dtype=float32>

#**Sentiment Analysis - GRU**

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
df_train = pd.read_csv('train.csv')

In [6]:
df_train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [7]:
train_tweets = df_train['tweet']

In [8]:
train_tweets[1]

"@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked"

In [9]:
import re
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'too', 'own', 'll', "shan't", 'y', 'have', "should've", 'off', 'hers', 'with', 'on', 'but', 'been', 'her', 'over', 'these', 'ain', 'them', 'haven', 'me', 'against', 'yourselves', 'again', 'out', 'doing', 'during', 'herself', 'do', 'will', 'such', 'why', 'than', 'by', 'did', 'as', 'can', 'into', "wasn't", "you'll", 'myself', 'my', 'in', 'd', 'which', 'through', "weren't", 'further', 'itself', 'we', 'ours', 'that', 'because', 'wouldn', 'what', 'm', 'were', "didn't", "isn't", 'yours', 'who', 'how', 'himself', 'doesn', 'between', 'or', 'being', 'you', 'themselves', 'and', 'their', 'if', "don't", 'a', 'very', 'until', 'nor', 'theirs', 'when', 'all', 'our', 'only', 'so', 'those', 'below', 'she', "doesn't", "it's", 'yourself', 'while', 'after', 'is', "that'll", "shouldn't", 'some', 'are', 'any', 'i', 'at', 'for', 'isn', "you'd", 've', 'to', "needn't", 'don', 'mightn', 'more', 'should', 'ourselves', 'has', "hadn't", "aren't", 'then', 'o', 'its', 'there', "you're", 'having', 'where', 'had', 'm

In [13]:
def clean_text(text):
  text = re.sub(r'@\w+',' ',text)
  text = re.sub(r'[^a-zA-Z]',' ',text)
  text = re.sub(r"what's", "what is ", text)
  text = re.sub(r"\'s", " ", text)
  text = re.sub(r"\'ve", " have ", text)
  text = re.sub(r"can't", "cannot ", text)
  text = re.sub(r"n't", " not ", text)
  text = re.sub(r"i'm", "i am ", text)
  text = re.sub(r"\'re", " are ", text)
  text = re.sub(r"\'d", " would ", text)
  text = re.sub(r"\'ll", " will ", text) 
  text = re.sub(r'ur',' your ', text) 
  text = text.lower()
  text = word_tokenize(text)
  
  text = [word for word in text if not word in stop_words if len(word) > 2]

  return text


In [14]:
df_train['clean tweets'] = df_train['tweet'].apply(clean_text)

In [15]:
df_train.head()

,id,label,tweet,clean tweets
0,1,0,@user when a father is dysfunctional and is s...,"[father, dysfunctional, selfish, drags, kids, ..."
1,2,0,@user @user thanks for #lyft credit i can't us...,"[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0,bihday your majesty,"[bihday, majesty]"
3,4,0,#model i love u take with u all the time in ...,"[model, love, take, time]"
4,5,0,factsguide: society now #motivation,"[factsguide, society, motivation]"


In [16]:
a = df_train['tweet'][1]

In [17]:
print(clean_text(a))

['thanks', 'lyft', 'credit', 'use', 'cause', 'offer', 'wheelchair', 'vans', 'pdx', 'disapointed', 'getthanked']


In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
clean_tweets = df_train['clean tweets']

In [20]:
import tensorflow
from tensorflow.keras.preprocessing.text import one_hot

In [21]:
clean_tweets = [' '.join(x) for x in clean_tweets]
print(clean_tweets[:10])

['father dysfunctional selfish drags kids dysfunction run', 'thanks lyft credit use cause offer wheelchair vans pdx disapointed getthanked', 'bihday majesty', 'model love take time', 'factsguide society motivation', 'huge fan fare big talking leave chaos pay disputes get allshowandnogo', 'camping tomorrow danny', 'next school year year exams think school exams hate imagine actorslife revolutionschool girl', 'love land allin cavs champions cleveland clevelandcavaliers', 'welcome']


In [22]:
maxVocab = 10000
clean_tweets_onehot = [one_hot(x,maxVocab) for x in clean_tweets]
clean_tweets_onehot[:5]

[[7356, 4441, 9900, 8750, 2795, 7191, 425],
 [17, 7444, 3746, 4371, 9982, 1442, 8417, 4624, 529, 7226, 26],
 [631, 7151],
 [4899, 657, 1386, 4771],
 [4324, 4753, 6367]]

In [23]:
maxLen = -1
for doc in clean_tweets_onehot:
  if len(doc) > maxLen:
    maxLen = len(doc)
print(maxLen)

19


In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
clean_tweets_pad = pad_sequences(clean_tweets_onehot,maxLen,padding='post')
clean_tweets_pad[:5]

array([[7356, 4441, 9900, 8750, 2795, 7191,  425,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [  17, 7444, 3746, 4371, 9982, 1442, 8417, 4624,  529, 7226,   26,
           0,    0,    0,    0,    0,    0,    0,    0],
       [ 631, 7151,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [4899,  657, 1386, 4771,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [4324, 4753, 6367,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32)

In [25]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model

In [57]:
input = layers.Input(shape=(maxLen,), dtype='float64')
word_embedding = layers.Embedding(maxVocab,128,input_length=maxLen)(input)
x = layers.GRU(32,return_sequences=True,)(word_embedding)
x = layers.GRU(64)(x)
output = layers.Dense(1,activation='sigmoid')(x)
model = Model(input,output)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [27]:
Y_train = df_train['label'].values
Y_train[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [59]:
H = model.fit(clean_tweets_pad,Y_train,batch_size=32,epochs=5,validation_split=0.2)

Epoch 1/5
541/541 [==============================] - 9s 17ms/step - loss: 0.0694 - accuracy: 0.9778 - val_loss: 0.0736 - val_accuracy: 0.9750
Epoch 2/5
541/541 [==============================] - 9s 16ms/step - loss: 0.0312 - accuracy: 0.9914 - val_loss: 0.0916 - val_accuracy: 0.9734
Epoch 3/5
541/541 [==============================] - 9s 16ms/step - loss: 0.0148 - accuracy: 0.9958 - val_loss: 0.1104 - val_accuracy: 0.9697
Epoch 4/5
541/541 [==============================] - 9s 16ms/step - loss: 0.0074 - accuracy: 0.9977 - val_loss: 0.1539 - val_accuracy: 0.9676
Epoch 5/5
541/541 [==============================] - 9s 16ms/step - loss: 0.0056 - accuracy: 0.9985 - val_loss: 0.1570 - val_accuracy: 0.9664


In [30]:
df_test = pd.read_csv('test.csv')

In [31]:
df_test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [32]:
df_test['clean tweets'] = df_test['tweet'].apply(clean_text)
df_test.head()

,id,tweet,clean tweets
0,31963,#studiolife #aislife #requires #passion #dedic...,"[studiolife, aislife, requires, passion, dedic..."
1,31964,@user #white #supremacists want everyone to s...,"[white, supremacists, want, everyone, see, new..."
2,31965,safe ways to heal your #acne!! #altwaystohe...,"[safe, ways, heal, acne, altwaystoheal, health..."
3,31966,is the hp and the cursed child book up for res...,"[sed, child, book, reservations, already, yes,..."
4,31967,"3rd #bihday to my amazing, hilarious #nephew...","[bihday, amazing, hilarious, nephew, eli, ahmi..."


In [66]:
clean_tweets_test = df_test['clean tweets']
clean_tweets_test = [' '.join(x) for x in clean_tweets_test]
clean_tweets_test_onehot = [one_hot(x,maxVocab) for x in clean_tweets_test]
clean_tweets_test_pad = pad_sequences(clean_tweets_test_onehot,maxLen,padding='post')

In [67]:
clean_tweets_test_pad[:5]

array([[8676, 4833, 4562, 8724, 9542, 2170, 4749, 9608,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [1698, 1281, 2661, 3127, 8548, 8199, 2488, 5624,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [1589, 8916, 9023, 9174, 1779, 3872, 2235,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [8801, 9532, 1299, 7643, 3329, 8137, 1176, 5883, 9518,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [ 631, 8505, 8320, 1844, 7490, 1308,  659, 1782, 9797, 7373,    0,
           0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32)

In [68]:
Y_pred = model.predict(clean_tweets_test_pad,batch_size=32)

In [69]:
Y_pred_f = list(map(lambda x: 1 if x > 0.5 else 0, Y_pred))

In [70]:
Y_pred_f[:300]

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [42]:
Y_train = Y_train.astype('float')

In [53]:
Y_train[:20]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 0.])

In [65]:
model.evaluate(clean_tweets_pad,Y_train)

676/676 [==============================] - 2s 3ms/step - loss: 0.0338 - accuracy: 0.9926


[0.03379938378930092, 0.9926395416259766]

In [71]:
df_test['label'] = Y_pred_f
df_test.head()

,id,tweet,clean tweets,label
0,31963,#studiolife #aislife #requires #passion #dedic...,"[studiolife, aislife, requires, passion, dedic...",0
1,31964,@user #white #supremacists want everyone to s...,"[white, supremacists, want, everyone, see, new...",1
2,31965,safe ways to heal your #acne!! #altwaystohe...,"[safe, ways, heal, acne, altwaystoheal, health...",0
3,31966,is the hp and the cursed child book up for res...,"[sed, child, book, reservations, already, yes,...",0
4,31967,"3rd #bihday to my amazing, hilarious #nephew...","[bihday, amazing, hilarious, nephew, eli, ahmi...",0


In [72]:
df_test.drop(['tweet','clean tweets'],axis=1,inplace=True)
df_test.head()

,id,label
0,31963,0
1,31964,1
2,31965,0
3,31966,0
4,31967,0


In [73]:
pd.DataFrame(df_test).to_csv('submissionGRU.csv',columns=['id','label'],index=None)